This code generates a csv for a 5% of each the countries in the WDPA data. 
This creates the template for the manual data collection (for checking data entry) of the costs.

In [ ]:
import geopandas as gpd
import pandas as pd

# Imports PA dataset
wdpa = gpd.read_file("../../data/WDPA/raw/WDPA_Jan2023_Public.gdb")

In [2]:
##############
# WARNING !!!!!!!!!!!
# If you rerun this code, it will overwrite the current data from the manual data collection.
##############

import numpy as np
from pathlib import Path 

# Set Seed 
import random
random.seed(12345)

# Create new variable of ISO3 to desginate if a park is international (e.g., multiple ISO3 codes)
wdpa["ISO3_Int"] = np.where(wdpa["ISO3"].str.contains(";"), "INT", wdpa["ISO3"])

# Get list of unique country ISO3 codes 
list_countries = set(wdpa["ISO3_Int"])

# Loop through countries 
for country in list_countries: 
    # Subset for country
    wdpa_country = wdpa[wdpa["ISO3_Int"] == country]

    # Only keep necessary info
    wdpa_country = pd.DataFrame(
                wdpa_country[["WDPAID", "WDPA_PID", "NAME", 
                        "ORIG_NAME", "DESIG_ENG", "DESIG_TYPE", 
                        "MARINE", "STATUS_YR", "OWN_TYPE", 
                        "MANG_AUTH", "ISO3"]]
    )
    # Add variables to collect info on costs
    added_vars = ["cost_established", "cost_maintainted"]
    for var in added_vars:
        wdpa_country[var] = ""
        wdpa_country[var + "_units"] = ""
        wdpa_country[var + "_source"] = ""

    # Proper subsampling of 5% of parks from every country to check.
    wdpa_country = wdpa_country.sample(frac = 0.05)
    
    # Export template for manual data entry
    filepath = Path('../../data/WDPA/costs/manual/costs_' + country + '_manual.csv')  
    filepath.parent.mkdir(parents = True, exist_ok = True)  
    wdpa_country.to_csv(filepath, index = False)  


In [2]:
# TODO: Count the len() so we know how many parks we are manually adding
import numpy as np

# Number of countries 
parks_checked = np.empty(list_countries, 2)

for country in list_countries: 
    # Subset for country
    wdpa_country = wdpa[wdpa["ISO3_Int"] == country] 
    # Proper subsampling of 5% of parks from every country to check.
    wdpa_country = wdpa_country.sample(frac = 0.05)
    parks_checked = np.vstack([parks_checked, [country, len(wdpa_country)]])

# How many parks are we manually checking in total and for each country (valuable for an appendix or reference for the future)
print(parks_checked) # TODO: Make this a table...


NameError: name 'list_countries' is not defined

In [ ]:
# TODO: Create a new script to compare the manual entry and the webscraping entry